In [1]:
# install and import package
!pip install  snownlp
!pip install  ggplot
import pandas
import requests
from snownlp import SnowNLP
from bs4 import BeautifulSoup


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 37.6 MB 4.6 MB/s eta 0:00:01    |██▊                             | 3.3 MB 833 kB/s eta 0:00:42     |███▊                            | 4.4 MB 833 kB/s eta 0:00:40     |█████▋                          | 6.6 MB 9.8 MB/s eta 0:00:04     |█████████▏                      | 10.8 MB 9.8 MB/s eta 0:00:03     |██████████                      | 11.7 MB 9.8 MB/s eta 0:00:03     |███████████████▋                | 18.3 MB 1.6 MB/s eta 0:00:13     |██████████████████              | 21.1 MB 1.6 MB/s eta 0:00:11     |████████████████████            | 23.4 MB 5.4 MB/s eta 0:00:03     |██████████████████████▋         | 26.5 MB 5.4 MB/s eta 0:00:03     |██████████████████████████▉     | 31.5 MB 5.7 MB/s eta 0:00:02     |█████████████████████████████   | 34.0 MB 5.7 MB/s eta 0:00:01
  

In [21]:
# 下載微博武漢肺炎相關貼文
def download_Cov_data(max):
    articles = []
    for i in range(1,max):
        res = requests.get('https://m.weibo.cn/api/container/getIndex?containerid=231522type%3D1%26q%3D%23%E6%AD%A6%E6%B1%89%E8%82%BA%E7%82%8E%23&page_type=searchall&page={}'.format(i))
        jd = res.json()
        articles.extend([rec['mblog'] for rec in jd['data']['cards'] if rec.get('mblog')])

    return pandas.DataFrame(articles)
# 撈1~50頁資料
df = download_Cov_data(50)


In [22]:
# 取出貼文資料
def parseArticle(e):
    soup = BeautifulSoup(e, 'html.parser')
    return soup.text

def parseQueryTime(e):
    return e.split('&qtime=')[1].replace("&","")

text_data=df['text'].map(parseArticle)
query_time_data=df['itemid'].map(parseQueryTime)

In [23]:
# 進行每編貼文的情緒分析

def getSentenment(e):
    s = SnowNLP(e)
    return s.sentiments

sentiments_data=text_data.map(getSentenment)

dict = {
    "查詢時間":query_time_data,
    "貼文內容": text_data,  
     "情緒分數": sentiments_data
       }

select_df = pandas.DataFrame(dict)

In [24]:
# 輸出分析結果至terminal
sorted_data= select_df.sort_values(by = '情緒分數',ascending = False)

# 輸出分析結果至export_dataframe.csv檔
# 情緒分數越大代表貼文內容越正面
sorted_data['情緒分數']=sorted_data['情緒分數'].map(lambda x:format(x,'.4%'))
sorted_data.to_csv (r'export_dataframe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path




In [25]:
sorted_data


,查詢時間,貼文內容,情緒分數
454,1580889478,#动物# #人与自然# #武汉肺炎# 动物、植物、人类等地球上一切生物都是大自然母亲的孩子关...,100.0000%
161,1580889461,去年此刻，我在感受整个世界的热闹，人群与我无关，欢笑也是，阴霾笼罩了我一年，整整一年啊，这一...,100.0000%
255,1580889466,【湖南发生H5N1禽流感‧4500只肉鸡发病死亡】2月1日，湖南省邵阳市双清区发生一起家禽H...,100.0000%
92,1580889457,我真的是一个眼窝子比较浅的人，很容易被感动，这些天每次看到有关武汉的视频眼泪都是吧嗒吧嗒的留...,100.0000%
36,1580889454,澳洲大火，中国疫情，东非蝗灾，开始想起来流浪地球的台词，“最初，没有人在意这场灾难，这不过是...,100.0000%
...,...,...,...
188,1580889462,#武汉肺炎# #全民口罩行动# #体温检测# #逆向思维# 进社区要量测体温，但离开社区的却...,0.0000%
160,1580889461,肺炎超话#武汉患者就医实情# @凤凰网视频 【患者亲述：#武汉邻市鄂州物资告急# 救护车不敢...,0.0000%
208,1580889463,肺炎超话#比利时新型肺炎疫情# @梨视频 【#比利时确诊首例新冠病毒肺炎患者#：#从武汉撤侨...,0.0000%
221,1580889464,#武汉肺炎# #健康一春# 普通家庭如何做好居家消毒？在疾病流行期间，外出回家后，应及时用洗...,0.0000%
